# Create RDF2Vec-based embeddings

In [ ]:
# Load required packages
import pandas as pd
from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.graphs import KG, Vertex
from pyrdf2vec.walkers import RandomWalker
from pyrdf2vec.samplers import UniformSampler
from pyrdf2vec.embedders import Word2Vec

In [ ]:
refset = "Teslovich"

In [ ]:
if refset == "Farashi":
    ref = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Input sets/Farashi/Farashi full 2000000 bp distance no pvalue filtering.csv")
    ensembl_ids = list(set(ref["gene_ids"]))

In [ ]:
if refset == "DeRycke":
    ref = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Input sets/DeRycke/DeRycke reference set.csv")
    ensembl_ids = list(set(ref["ENSEMBL"]))

In [ ]:
if refset == "Teslovich":
    ref = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Input sets/Teslovich/Teslovich reference set.csv")
    ensembl_ids = list(set(ref["ENSEMBL"]))

In [ ]:
# Load the triples
PPI = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Complete set of protein-protein interactions extracted from PathwayStudio.csv")
PPI = PPI[["ENSEMBL1", "predicate", "ENSEMBL2"]]

entities = list(set(PPI["ENSEMBL1"]).union(set(PPI["ENSEMBL2"])))

In [ ]:
ensembl_ids = list(set(ensembl_ids).intersection(set(entities)))

In [ ]:
URL = "http://pyRDF2Vec"
CUSTOM_KG = KG()

In [ ]:
for index, row in PPI.iterrows():
     subj = Vertex(URL + "#" + row["ENSEMBL1"])
     obj = Vertex(URL + "#" + row["ENSEMBL2"])
     pred = Vertex(URL + "#" + row["predicate"], predicate=True, vprev=subj, vnext=obj)
     CUSTOM_KG.add_walk(subj, pred, obj)

In [ ]:
# Use default settins, only increase number of cores being used
transformer = RDF2VecTransformer(walkers = [RandomWalker(2, None, UniformSampler(), n_jobs = 1)])
embeddings, literals = transformer.fit_transform(CUSTOM_KG, entities = [URL + "#" + x for x in ensembl_ids])

In [ ]:
features = pd.DataFrame(embeddings, columns = ["F"+str(i+1) for i in range(len(embeddings[1]))], index = ensembl_ids)
features.to_csv("~/git/Post-GWAS/RDF2vec/" + refset + " complete protein-protein embeddings.csv")                     